## 1. Análisis de componentes Principales

### 3.1. Estudie si es posible hacer transformaciones en las variables categóricas para incluirlas en el PCA, ¿valdrá la pena?

Transformaciones comunes para variables categóricas:
1.Codificación One-Hot (dummies):
   - Convierte cada categoría de una variable en una columna separada con valores binarios (0 o 1).
   - Ejemplo: Si tienes una variable "Color" con categorías "Rojo", "Verde" y "Azul", se crearán tres columnas: Color_Rojo, Color_Verde, Color_Azul.
   - Esto puede aumentar significativamente el número de características, lo cual podría dificultar la interpretación si el número de categorías es muy grande.

2.Codificación de etiquetas (Label Encoding):
   - Asigna un número único a cada categoría.
   - Ejemplo: Si "Rojo" = 1, "Verde" = 2, "Azul" = 3, la variable se convierte en una variable numérica.
   - Aunque esta es una opción más compacta, puede implicar que el modelo pueda interpretar las categorías como ordinales, 
     lo cual no es apropiado si no hay un orden natural entre las categorías.
3.Codificación Binaria:
   - Convierte las categorías a su representación binaria, lo cual puede ser útil si el número de categorías es grande y se desea mantener la información en un espacio más compacto.

¿Vale la pena hacer estas transformaciones?
   - Si las variables categóricas tienen un impacto significativo en la variabilidad de los datos, transformarlas puede proporcionar una representación más rica y útil para el PCA.

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Cargar los datos (suponiendo que ya tienes un DataFrame llamado 'df')
df = pd.read_csv('movies.csv')

# Verifica las columnas categóricas
print("Columnas categóricas:", df.select_dtypes(include=['object']).columns)

# Codificación One-Hot
df_encoded = pd.get_dummies(df, drop_first=True)  # drop_first=True evita multicolinealidad

# Ver los primeros registros después de la codificación
print(df_encoded.head())

# Estandarización de las variables numéricas
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_encoded)

# Mostrar la forma de los datos transformados
print("Forma de los datos escalados:", df_scaled.shape)



### 3.2. Estudie si es conveniente hacer un Análisis de Componentes Principales. Recuerde que puede usar el índice KMO y el test de esfericidad de Bartlett.

El PCA es útil cuando se busca reducir la dimensionalidad sin perder una cantidad significativa de información. Para evaluar si es conveniente realizar PCA, se deben utilizar dos pruebas: el índice KMO (Kaiser-Meyer-Olkin) y el test de esfericidad de Bartlett.


1.Prueba de Esfericidad de Bartlett:
   - Se utiliza para verificar si las variables están correlacionadas lo suficiente como para realizar un análisis factorial (PCA).
   - Hipótesis nula: La matriz de correlación es una matriz identidad (lo que indica que no hay correlación significativa).
   - Si el valor p de la prueba es bajo (generalmente < 0.05), se puede proceder con PCA porque las variables están correlacionadas de manera significativa.

2.Índice KMO (Kaiser-Meyer-Olkin):
   - Mide la adecuación de la muestra para el análisis factorial.
   - El valor KMO varía entre 0 y 1, siendo:
        - Valor KMO cercano a 1 indica que los datos son adecuados para PCA.
        - Valor KMO menor que 0.5 indica que no es recomendable realizar PCA.



In [ ]:
from scipy.stats import chi2
import numpy as np

def bartlett_test(df):
    # Calcular la matriz de correlación
    correlation_matrix = df.corr()
    n = len(df)  # Número de observaciones
    p = df.shape[1]  # Número de variables
    
    # Calcular el estadístico de Bartlett
    bartlett_stat = (n - 1 - (2 * p + 5) / 6) * np.log(np.linalg.det(correlation_matrix))
    
    # Grados de libertad
    df_bartlett = (p * (p - 1)) / 2
    p_value = 1 - chi2.cdf(bartlett_stat, df_bartlett)
    
    return bartlett_stat, p_value

# Ejecutar la prueba de Bartlett
bartlett_stat, p_value = bartlett_test(df_encoded)
print(f'Estadístico de Bartlett: {bartlett_stat}')
print(f'Valor p de Bartlett: {p_value}')



In [ ]:
from factor_analyzer import calculate_kmo

# Calcular el índice KMO
kmo_all, kmo_model = calculate_kmo(df_encoded)
print(f'Índice KMO: {kmo_model}')


### 3.3. Haga un análisis de componentes principales con las variables numéricas, discuta los resultados e interprete los componentes.

Una vez que se determina que es conveniente realizar PCA (basado en las pruebas anteriores), se procede con el análisis de los componentes principales utilizando solo las variables numéricas.


1.Pasos para realizar PCA:
   - Normalización de los datos: Si las variables numéricas tienen escalas diferentes, es necesario normalizarlas (por ejemplo, utilizando estandarización, para que tengan media 0 y desviación estándar 1).
   - Cálculo de la matriz de covarianza o correlación de las variables numéricas.
   - Extracción de los componentes principales: Usamos un algoritmo de descomposición, como la descomposición en valores singulares (SVD), para obtener los componentes principales.
   - Evaluación de la varianza explicada por cada componente principal:
      - Los componentes principales explican la mayor parte de la variabilidad de los datos. Se seleccionan los primeros componentes con la mayor varianza acumulada.
      - Criterio común: Seleccionar suficientes componentes para explicar entre el 70% y el 90% de la varianza acumulada.

2.Discusión de los Resultados:
   - Componentes principales: Los componentes principales que expliquen la mayor parte de la variabilidad de los datos serán los más relevantes.
   - Interpretación de los componentes:
      - Carga de las variables: Cada componente tendrá una serie de coeficientes (cargas) que indican la contribución de cada variable en ese componente.
      - Componente dominante: El primer componente generalmente será el más importante, y será una combinación lineal de las variables que más contribuyen a la variabilidad.
   - Ejemplo de interpretación:
      - Si el primer componente principal tiene una alta carga para las variables A, B y C, esto sugiere que estas variables están correlacionadas y explican gran parte de la variabilidad en los datos.
      - El segundo componente puede capturar una variabilidad ortogonal a la del primero, y tendrá una carga alta en otras variables que no se correlacionan con las del primer componente.


In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

# Realizar PCA
pca = PCA()
pca.fit(df_scaled)

# Varianza explicada por cada componente
explained_variance = pca.explained_variance_ratio_

# Mostrar la varianza explicada por cada componente
print(f'Varianza explicada por cada componente: {explained_variance}')

# Gráfico de varianza explicada
plt.plot(range(1, len(explained_variance) + 1), explained_variance, marker='o')
plt.title('Varianza explicada por cada componente principal')
plt.xlabel('Número de componente')
plt.ylabel('Varianza explicada')
plt.show()

# Selección de componentes: seleccionamos los que explican al menos 80% de la varianza
cumulative_variance = np.cumsum(explained_variance)
num_components = np.argmax(cumulative_variance >= 0.80) + 1  # Selección del número de componentes
print(f'Número de componentes seleccionados: {num_components}')

# Obtener los componentes principales
principal_components = pca.components_

# Mostrar los coeficientes principales
print('Coeficientes principales (cargas):')
print(principal_components)
